In [ ]:
import pandas as pd
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, activation='tanh', input_shape=input_shape))
    model.add(Dense(1))  
    model.compile(optimizer='adam', loss='mse')
    return model

In [ ]:
model = build_lstm_model(input_shape=(20, 1))  
model.fit(X_train, y_train, epochs=50, validation_split=0.2)

In [ ]:
model.save("modelo_lstm_temperatura.h5")